In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.nn.functional as F

In [2]:
BATCH_SIZE=512
EPOCHS=20
# Let Torch decide which device to use
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=False, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class MLP(nn.Module):
    # Initialize parameters
    def __init__(self, num_outputs, num_hiddens, lr=0.01):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(), 
            nn.LazyLinear(num_hiddens),
            nn.ReLU(),
            nn.LazyLinear(num_outputs)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [6]:
model = MLP(10, 10).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())

/home/yi/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 1.326467
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.775689
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.557540

Test set: Average loss: 0.4717, Accuracy: 8749/10000 (87%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.442232
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.361139
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.313894

Test set: Average loss: 0.3422, Accuracy: 9026/10000 (90%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.384454
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.417952
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.318887

Test set: Average loss: 0.3023, Accuracy: 9150/10000 (92%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.285957
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.303570
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.252664

Test set: Average loss: 0.2801, Accuracy: 9209/10000 (92%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.318085
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.220354
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.331808

Test 